In [4]:
import pandas as pd
from tensorflow import keras
from os import path
import pickle


In [38]:
from proteinbert.finetuning import encode_train_and_valid_sets, encode_dataset
from proteinbert import OutputType, OutputSpec, evaluate_by_len, load_pretrained_model

In [39]:
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, \
finetune, evaluate_by_len

from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [40]:
import wandb
from wandb.keras import WandbCallback

In [41]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [42]:
DATA_DIR = "../../data/"

In [43]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [44]:
pretrained_model_generator, input_encoder = load_pretrained_model("../../data/protein_bert/", "epoch_92400_sample_23500000.pkl")

In [45]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [10]:
wandb.init(project=f"April_finetune", entity="kvetab")

wandb: Currently logged in as: kvetab (use `wandb login --relogin` to force relogin)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


# Split 4a

In [11]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/crossval/chen_4_a.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/crossval/chen_4_b.csv"), index_col=0)
train_data.head()

,Antibody_ID,heavy,light,Y,cluster
535,2g60,EVQLQQSGGELAKPGASVKMSCKSSGYTFTAYAIHWAKQAAGAGLE...,DVLMTQAPLTLPVSLGDQASISCRSSQAIVHANGNTYLEWYLQKPG...,0,911
1120,4ffy,QVQLLQPGAELVKPGASMKLSCKASGYTFTNWWMHWVRLRPGRGLE...,NIVLTQSPASLAVSLGQRATISCRASESVDHYGNSFIYWYQQKPGQ...,0,478
408,1w72,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,SYVLTQPPSVSVAPGQTARITCGGNNIGSRSVHWYQQKPGQAPVLV...,0,836
669,2xwt,EVQLVQSGAEVKKPGQSLKISCKASGYSLTDNWIGWVRQKPGKGLE...,QSVLTQPPSVSAAPGQKVTISCSGSSSDIGSNYVSWYQQFPGTAPK...,0,545
568,2ipt,QVTLKESGPGILKPSQTLSLTCSFSGFSLSTSGMGVGWIRQPSGKG...,DVLMTQTPLSLPVSLGDQASISCRSSQSIVHSNGNTYLEWYLQKPG...,0,55


In [12]:
train_data["seq"] = train_data["heavy"] + train_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

In [13]:
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 3, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True),
    WandbCallback()
]

In [14]:
epoch_num = 50
batch_size = 128
learning_rate = 1e-4

In [15]:
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }

In [19]:
##### Validating using test data!!! #####

In [17]:
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], test_data['seq'], test_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_04_09-11:29:56] Training set: Filtered out 0 of 637 (0.0%) records of lengths exceeding 510.
[2022_04_09-11:29:56] Validation set: Filtered out 0 of 654 (0.0%) records of lengths exceeding 510.
[2022_04_09-11:29:56] Training with frozen pretrained layers...


2022-04-09 11:29:59.899855: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-09 11:30:03.060697: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:41:00.0, compute capability: 7.5
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-04-09 11:30:05.411634: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are en

Epoch 1/50


2022-04-09 11:30:17.126370: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 7605


5/5 [==============================] - 15s 770ms/step - loss: 1.0164 - val_loss: 0.9833


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/50
5/5 [==============================] - 1s 233ms/step - loss: 0.6325 - val_loss: 0.6123
Epoch 3/50
5/5 [==============================] - 1s 231ms/step - loss: 0.5371 - val_loss: 0.5515
Epoch 4/50
5/5 [==============================] - 1s 235ms/step - loss: 0.4412 - val_loss: 0.5147
Epoch 5/50
5/5 [==============================] - 1s 233ms/step - loss: 0.4427 - val_loss: 0.5044
Epoch 6/50
5/5 [==============================] - 1s 233ms/step - loss: 0.4353 - val_loss: 0.5586
Epoch 7/50
5/5 [==============================] - 1s 232ms/step - loss: 0.4335 - val_loss: 0.4875
Epoch 8/50
5/5 [==============================] - 1s 232ms/step - loss: 0.3942 - val_loss: 0.5111
Epoch 9/50
5/5 [==============================] - 1s 232ms/step - loss: 0.3829 - val_loss: 0.4740
Epoch 10/50
5/5 [==============================] - 1s 233ms/step - loss: 0.3746 - val_loss: 0.4777
Epoch 11/50
5/5 [==============================] - 1s 234ms/step - loss: 0.3685 - val_loss: 0.4681
Epoch 12/50
5/5 [=

In [18]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)

In [20]:
print('Test-set performance:')
display(results)

print('Confusion matrix:')
display(confusion_matrix)

Test-set performance:


,# records,AUC
Model seq len,,
512,654,0.798584
All,654,0.798584


Confusion matrix:


,0,1
0,471,29
1,99,55


In [21]:
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

0.46218487394957986

In [22]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/2022_04_09_01"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-04-09 11:32:18.361248: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_04_09_01/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_04_09_01/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [33]:
patience = [(3,3), (4,3), (6,3), (8,4)]
learning_rate = [1e-5, 5e-5, 1e-4, 5e-4]
prepro = ["scaling"]

In [34]:
def finetune_by_settings(patience, lr, project_name):
    wandb.init(project=project_name, entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)
    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience = patience[1], factor = 0.25, min_lr = 1e-07, verbose = 1),
        keras.callbacks.EarlyStopping(patience = patience[0], restore_best_weights = True),
        WandbCallback()
    ]
    epoch_num = 100
    batch_size = 128
    wandb.config = {
          "learning_rate": lr,
          "epochs": epoch_num * 2,
          "batch_size": batch_size
        }
    print(type(lr))
    print(lr)
    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], test_data['seq'], test_data['Y'], \
            seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = lr, begin_with_frozen_pretrained_layers = True, \
            lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 512, final_lr = lr / 10, callbacks = training_callbacks)
    
    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
            start_seq_len = 512, start_batch_size = 32)
    print('Confusion matrix:')
    display(confusion_matrix)
    fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
    f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
    print(f1)
    
    mod = model_generator.create_model(seq_len = 512)
    mod.save(path.join(DATA_DIR, f"protein_bert/4b/2022_04_09_{patience[0]}_{patience[1]}_{lr}"))

In [33]:
for pat in patience:
    for lr in learning_rate:
        finetune_by_settings(pat, lr, "Split 4a")

wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
1e-05
[2022_04_09-16:39:18] Training set: Filtered out 0 of 637 (0.0%) records of lengths exceeding 510.
[2022_04_09-16:39:18] Validation set: Filtered out 0 of 654 (0.0%) records of lengths exceeding 510.
[2022_04_09-16:39:18] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
5/5 [==============================] - 10s 713ms/step - loss: 0.9282 - val_loss: 0.6173


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


...

,0,1
0,491,9
1,124,30


0.31088082901554404


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_04_09_8_4_0.0005/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_04_09_8_4_0.0005/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [36]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/crossval/chen_4_b.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/crossval/chen_4_a.csv"), index_col=0)
train_data["seq"] = train_data["heavy"] + train_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

for pat in patience:
    for lr in learning_rate:
        finetune_by_settings(pat, lr, "Split 4b")

wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
1e-05
[2022_04_09-17:22:57] Training set: Filtered out 0 of 654 (0.0%) records of lengths exceeding 510.
[2022_04_09-17:22:57] Validation set: Filtered out 0 of 637 (0.0%) records of lengths exceeding 510.
[2022_04_09-17:22:57] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
6/6 [==============================] - 54s 581ms/step - loss: 0.8790 - val_loss: 0.4708


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


...

,0,1
0,458,71
1,66,42


0.38009049773755654


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/4b/2022_04_09_8_4_0.0005/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/4b/2022_04_09_8_4_0.0005/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# 5x2cv

In [35]:
def finetune_by_settings_and_data(patience, lr, project_name, train_data, test_data, dir_name):
    wandb.init(project=project_name, entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)
    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience = patience[1], factor = 0.25, min_lr = 1e-07, verbose = 1),
        keras.callbacks.EarlyStopping(patience = patience[0], restore_best_weights = True),
        WandbCallback()
    ]
    epoch_num = 100
    batch_size = 128
    wandb.config = {
          "learning_rate": lr,
          "epochs": epoch_num * 2,
          "batch_size": batch_size
        }
    print(type(lr))
    print(lr)
    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], test_data['seq'], test_data['Y'], \
            seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = lr, begin_with_frozen_pretrained_layers = True, \
            lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 512, final_lr = lr / 10, callbacks = training_callbacks)
    
    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
            start_seq_len = 512, start_batch_size = 32)
    print('Confusion matrix:')
    display(confusion_matrix)
    fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
    f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
    print(f1)
    
    mod = model_generator.create_model(seq_len = 512)
    mod.save(path.join(DATA_DIR, f"protein_bert/{dir_name}/2022_04_09_{patience[0]}_{patience[1]}_{lr}"))

In [38]:
for seed in [18, 27, 36, 42]: # 4
    train = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_a.csv"), index_col=0)
    test = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_b.csv"), index_col=0)
    train["seq"] = train["heavy"] + train["light"]
    test["seq"] = test["heavy"] + test["light"]
    for pat in patience:
        for lr in learning_rate:
            finetune_by_settings_and_data(pat, lr, f"Split {seed}a", train, test, f"{seed}a")
            finetune_by_settings_and_data(pat, lr, f"Split {seed}b", test, train, f"{seed}b")

epoch,▁▁▁▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇███▁▁▂▂▂▂▃▃▃▄▄▄▅▅▁
loss,█▆▅▆▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▄▃▃▃▃▂▂▂▂▂▁▁▁▂
lr,█████▃▃▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▃▄▂▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▂▂▁
best_epoch,9
best_val_loss,0.3859
epoch,0
loss,0.29658
lr,5e-05
val_loss,0.40332


wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
1e-05
[2022_04_09-17:59:32] Training set: Filtered out 0 of 615 (0.0%) records of lengths exceeding 510.
[2022_04_09-17:59:32] Validation set: Filtered out 0 of 676 (0.0%) records of lengths exceeding 510.
[2022_04_09-17:59:32] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
5/5 [==============================] - 11s 755ms/step - loss: 0.9407 - val_loss: 0.5439


...

,0,1
0,481,4
1,134,14


0.1686746987951807


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/42b/2022_04_09_8_4_0.0005/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/42b/2022_04_09_8_4_0.0005/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# Load Model

In [23]:
#from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn import metrics
import os
import csv

In [43]:
seq_len = 512
encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')

[2022_04_10-09:23:52] Test set: Filtered out 0 of 637 (0.0%) records of lengths exceeding 510.


In [22]:
test_X, test_Y, test_sample_weigths = encoded_test_set

In [47]:
tap_data = pd.read_csv(path.join(DATA_DIR, "tap/TAP_data.csv"))
tap_data.head()

,Antibody_ID,heavy,light,CDR_length,PSH,PPC,PNC,SFvCSP,Y
0,Abagovomab,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLD...,DIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQL...,46,129.7603,0.0000,0.0000,16.32,1
1,Abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,45,115.9106,0.0954,0.0421,-3.10,1
2,Abrilumab,QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKL...,45,109.6995,0.0000,0.8965,-4.00,1
3,Actoxumab,QVQLVESGGGVVQPGRSLRLSCAASGFSFSNYGMHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQHKPGKAPKL...,49,112.6290,0.0000,1.1247,3.10,1
4,Adalimumab,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNYLAWYQQKPGKAPKL...,48,111.2512,0.0485,1.1364,-19.50,1


In [48]:
tap_data["seq"] = tap_data["heavy"] +  tap_data["light"]

In [49]:
encoded_tap_set = encode_dataset(tap_data["seq"], tap_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'TAP set')

[2022_04_15-12:44:22] TAP set: Filtered out 0 of 241 (0.0%) records of lengths exceeding 510.


In [50]:
tap_X, tap_Y, tap_sample_weigths = encoded_tap_set

In [23]:
model_path = path.join(DATA_DIR, "protein_bert/2022_03_30__05.pkl")
model = keras.models.load_model(model_path)

In [24]:
y_pred = model.predict(test_X, batch_size=32)
y_pred_classes = (y_pred >= 0.5)
f1_score(test_Y, y_pred_classes)

0.6486486486486486

In [29]:
y_pred = model.predict(tap_X, batch_size=32)
y_pred_classes = (y_pred >= 0.5)
f1_score(tap_Y, y_pred_classes)

0.34285714285714286

In [30]:
accuracy_score(tap_Y, y_pred_classes)

0.23651452282157676

In [25]:
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
seq_len = 512
encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')
test_X, test_Y, test_sample_weigths = encoded_test_set

[2022_04_15-11:35:42] Test set: Filtered out 0 of 614 (0.0%) records of lengths exceeding 510.


In [27]:
def test_model(model_name, dir_name, x_test, y_test, x_tap, y_tap):
    model_path = path.join(DATA_DIR, f"protein_bert/{dir_name}/{model_name}")
    model = keras.models.load_model(model_path)
    parts = model_name[11:].split("_")
    patience_stop = parts[0]
    patience_lr = parts[1]
    lr = parts[2]
    
    y_pred = model.predict(x_test, batch_size=32)
    y_pred_classes = (y_pred >= 0.5)
    #print(f"Model {model_name}")
    #print(f"Test F1: {f1}")
    metric_dict = {
        "f1": float(metrics.f1_score(y_test, y_pred_classes)),
        "acc": float(metrics.accuracy_score(y_test, y_pred_classes)),
        "mcc": float(metrics.matthews_corrcoef(y_test, y_pred_classes)),
        "auc": float(metrics.roc_auc_score(y_test, y_pred_classes)),
        "precision": float(metrics.precision_score(y_test, y_pred_classes)),
        "recall": float(metrics.recall_score(y_test, y_pred_classes))
    }
    filename = path.join(DATA_DIR, f"evaluations/protein_bert/{dir_name}/{model_name}_preds.csv")
    str_preds = [str(int(pred)) for pred in y_pred_classes]
    with open(filename, "wt") as f:
        f.write(",".join(str_preds) + "\n")
        
    filename_sum = path.join(DATA_DIR, f"evaluations/protein_bert/{dir_name}/all.csv")
    line = ["protein_bert", patience_stop, patience_lr, lr, metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"]]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)
    
    
    y_pred = model.predict(x_tap, batch_size=32)
    y_pred_classes = (y_pred >= 0.5)
    acc = accuracy_score(y_tap, y_pred_classes)
    metric_dict = {
        "f1": float(metrics.f1_score(y_tap, y_pred_classes)),
        "acc": float(metrics.accuracy_score(y_tap, y_pred_classes)),
        "mcc": float(metrics.matthews_corrcoef(y_tap, y_pred_classes)),
        "auc": float(metrics.roc_auc_score(y_tap, y_pred_classes)),
        "precision": float(metrics.precision_score(y_tap, y_pred_classes)),
        "recall": float(metrics.recall_score(y_tap, y_pred_classes))
    }
        
    filename_sum = path.join(DATA_DIR, f"evaluations/protein_bert/{dir_name}/tap.csv")
    line = ["protein_bert", patience_stop, patience_lr, lr, metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"]]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)

In [57]:
seed = 4
split = "a"
train_data = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_a.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_b.csv"), index_col=0)
train_data["seq"] = train_data["heavy"] + train_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')
test_X, test_Y, test_sample_weigths = encoded_test_set

test_model("2022_04_09_3_3_0.0001", "4a", test_X, test_Y, tap_X, tap_Y)

[2022_04_10-09:53:46] Test set: Filtered out 0 of 654 (0.0%) records of lengths exceeding 510.


In [28]:
seeds = [4, 18, 27, 36, 42]
seeds = [36, 42]

In [16]:
seq_len = 512

In [29]:
for seed in seeds:
    train_data = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_a.csv"), index_col=0)
    test_data = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_b.csv"), index_col=0)
    train_data["seq"] = train_data["heavy"] + train_data["light"]
    test_data["seq"] = test_data["heavy"] + test_data["light"]
    encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')
    test_X, test_Y, test_sample_weigths = encoded_test_set
    encoded_train_set = encode_dataset(train_data["seq"], train_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Train set')
    train_X, train_Y, test_sample_weigths = encoded_train_set
    
    for split in ["a", "b"]:
        for model in os.listdir(path.join(DATA_DIR, f"protein_bert/{seed}{split}")):
            if model.startswith("2022_04_09"):
                if split == "a":
                    test_model(model, f"{seed}{split}", test_X, test_Y, tap_X, tap_Y)
                else:
                    test_model(model, f"{seed}{split}", train_X, train_Y, tap_X, tap_Y)
                    
                

[2022_04_15-11:36:29] Test set: Filtered out 0 of 614 (0.0%) records of lengths exceeding 510.
[2022_04_15-11:36:29] Train set: Filtered out 0 of 677 (0.0%) records of lengths exceeding 510.
[2022_04_15-11:41:15] Test set: Filtered out 0 of 658 (0.0%) records of lengths exceeding 510.
[2022_04_15-11:41:15] Train set: Filtered out 0 of 633 (0.0%) records of lengths exceeding 510.


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modi

# Full train data

In [31]:
from sklearn.model_selection import train_test_split

In [88]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test.csv"), index_col=0)
train_data["seq"] = train_data["heavy"] + train_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=39)

In [85]:
train_data_old = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data_old.csv"), index_col=0)
test_data_old = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test_data_old.csv"), index_col=0)
valid_data_old = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data_old.csv"), index_col=0)
train_data_old["seq"] = train_data_old["heavy"] + train_data_old["light"]
test_data_old["seq"] = test_data_old["heavy"] + test_data_old["light"]
valid_data_old["seq"] = valid_data_old["heavy"] + valid_data_old["light"]

In [92]:
for pat in patience:
    for lr in learning_rate:
        finetune_by_settings_and_data(pat, lr, f"Full old", train_data_old, valid_data_old, f"old_full")

epoch,▁▁▂▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇██▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇
loss,█▅▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
lr,███████▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▂▂▂▂▂▁▂▂▁▁▂▁▂▁▁▁▁▁▁▁▂▁▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁
best_epoch,13
best_val_loss,0.45486
epoch,0
loss,0.32702
lr,1e-05
val_loss,0.46158


wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
1e-05
[2022_04_15-14:37:59] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_04_15-14:37:59] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_04_15-14:37:59] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
11/11 [==============================] - 10s 286ms/step - loss: 0.8872 - val_loss: 0.7528


...

,0,1
0,93,1
1,18,8


0.45714285714285713


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/old_full/2022_04_09_8_4_0.0005/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/old_full/2022_04_09_8_4_0.0005/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [51]:
encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')

[2022_04_15-12:45:47] Test set: Filtered out 0 of 260 (0.0%) records of lengths exceeding 510.


In [52]:
test_X, test_Y, test_sample_weigths = encoded_test_set

In [93]:
encoded_test_set_old = encode_dataset(test_data_old["seq"], test_data_old["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set old')
test_X_old, test_Y_old, test_sample_weigths_old = encoded_test_set_old

[2022_04_15-15:27:54] Test set old: Filtered out 0 of 119 (0.0%) records of lengths exceeding 510.


In [54]:
for model in os.listdir(path.join(DATA_DIR, f"protein_bert/full")):
    if model.startswith("2022_04_09"):
        test_model(model, f"full", test_X, test_Y, tap_X, tap_Y)

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [95]:
for model in os.listdir(path.join(DATA_DIR, f"protein_bert/old_full")):
    if model.startswith("2022_04_09"):
        test_model(model, f"old_full", test_X_old, test_Y_old, tap_X, tap_Y)

In [59]:
model_name = "2022_04_09_6_3_0.0001"
model_path = path.join(DATA_DIR, f"protein_bert/full/{model_name}")
model = keras.models.load_model(model_path)
parts = model_name[11:].split("_")
patience_stop = parts[0]
patience_lr = parts[1]
lr = parts[2]

y_pred = model.predict(test_X, batch_size=32)
y_pred_classes = (y_pred >= 0.5)

In [61]:
metrics.f1_score(test_Y, y_pred_classes)

0.15384615384615383

In [62]:
metrics.confusion_matrix(test_Y, y_pred_classes)

array([[200,   6],
       [ 49,   5]])

In [80]:
len(train_data)

1291

In [89]:
wandb.init(project="Easter", entity="kvetab")
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
    get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 3, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True),
    WandbCallback()
]
epoch_num = 100
batch_size = 128
lr = 1e-4
wandb.config = {
      "learning_rate": lr,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
print(type(lr))
print(lr)
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = lr, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 512, final_lr = lr / 10, callbacks = training_callbacks)


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█▁▁▂▃▃▃▄▅▅▅▆▇▁
loss,█▄▃▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
lr,████████████▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▃▂▁▂▃▁▄▃▂▁▂▃▂▂▃▁▁▁▂▁▂▁▂▁▁
best_epoch,5
best_val_loss,0.45819
epoch,0
loss,0.35927
lr,1e-05
val_loss,0.48604


wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
0.0001
[2022_04_15-14:33:47] Training set: Filtered out 0 of 1291 (0.0%) records of lengths exceeding 510.
[2022_04_15-14:33:48] Validation set: Filtered out 0 of 130 (0.0%) records of lengths exceeding 510.
[2022_04_15-14:33:48] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
11/11 [==============================] - 10s 281ms/step - loss: 0.7999 - val_loss: 0.8898


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/100
11/11 [==============================] - 1s 108ms/step - loss: 0.5859 - val_loss: 0.6600
Epoch 3/100
11/11 [==============================] - 1s 109ms/step - loss: 0.5152 - val_loss: 0.5327
Epoch 4/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4500 - val_loss: 0.5070
Epoch 5/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4278 - val_loss: 0.4914
Epoch 6/100
11/11 [==============================] - 1s 110ms/step - loss: 0.4082 - val_loss: 0.5343
Epoch 7/100
11/11 [==============================] - 1s 107ms/step - loss: 0.4290 - val_loss: 0.5193
Epoch 8/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4161 - val_loss: 0.5199

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 9/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4100 - val_loss: 0.4779
Epoch 10/100
11/11 [==============================] - 1s 111ms/step - loss: 0.4006 - val_loss: 0.4866
Epoch 11/

11/11 [==============================] - 11s 418ms/step - loss: 0.4045 - val_loss: 0.4907
Epoch 2/100
11/11 [==============================] - 3s 247ms/step - loss: 0.3969 - val_loss: 0.4714
Epoch 3/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3894 - val_loss: 0.4817
Epoch 4/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3789 - val_loss: 0.4876
Epoch 5/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3825 - val_loss: 0.4609
Epoch 6/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3693 - val_loss: 0.4934
Epoch 7/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3705 - val_loss: 0.5067
Epoch 8/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3528 - val_loss: 0.4917

Epoch 00008: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 9/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3563 - val_loss: 0.4596
Epoch 10/100
11/11 [=

11/11 [==============================] - 11s 418ms/step - loss: 0.3270 - val_loss: 0.4616


In [90]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)

Confusion matrix:


,0,1
0,104,2
1,21,3


In [91]:
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

0.20689655172413793

In [83]:
len(train_data)

1291

In [86]:
len(test_data_old)

119

In [87]:
len(test_data)

156